<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Figure 1: Observed and modelled forest risks

_Authors: Oriana Chegwidden and Jeremy Freeman_

The methods below conduct the analyses to recreate Figure 1 included the
manuscript <<< insert doi >>>. They draw from a model data produced by running
`../../../scripts/fire.py` to create a the file located at
`az:carbonplan-scratch/data/fire.zarr`.


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forest_risks import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forest_risks.utils import get_store
import altair as alt
import rioxarray
from carbonplan.data import cat
alt.data_transformers.disable_max_rows()

### Initialize the dictionary where you'll store all of your datasets.


In [ ]:
ds_dict = {}

## Load in the fire data


In [ ]:
coarsen = 4
store = "az"
tlim = ("1984", "2018")

#### Historical simulation


In [ ]:
historical_fire = xr.open_zarr(
    get_store("carbonplan-scratch", "data/fire_historical_v3.zarr")
)

In [ ]:
fire_mask = ~np.isnan(historical_fire.historical.isel(time=0).drop("time"))

In [ ]:
ds_dict["Fire_historical"] = (
    xr.open_zarr(
        get_store("carbonplan-scratch", "data/fire_historical_v3.zarr")
    )
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .mean(dim="year")
).compute()

#### Observations


In [ ]:
mask = (
    load.nlcd(store=store, year=2001).sel(band=[41, 42, 43, 90]).sum("band")
    > 0.25
).astype("float")

In [ ]:
ds_dict["Fire_observed"] = load.mtbs(
    store=store, coarsen=coarsen, tlim=tlim, mask=mask
)
ds_dict["Fire_observed"] = (
    ds_dict["Fire_observed"]
    .assign_coords(
        {"x": ds_dict["Fire_historical"].x, "y": ds_dict["Fire_historical"].y}
    )
    .assign_coords(
        {
            "lat": ds_dict["Fire_historical"].lat,
            "lon": ds_dict["Fire_historical"].lon,
        }
    )
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .mean(dim="year")
).compute()

### Load in insect and drought data


In [ ]:
def load_bill(url, lat_coords, lon_coords, x_coords, y_coords):
    target = cat.nlcd.raster.to_dask()
    source = xr.open_rasterio(url)
    source = source.where(source > -1)
    ds = source.rio.reproject_match(target)
    ds = (
        ds.where(ds > -1)
        .coarsen(x=coarsen, y=coarsen, boundary="trim")
        .mean()
        .sel(band=1)
    )
    # make sure that the coordinates are *exactly* aligned- otherwise you'll have
    # pesky plotting peculiarities
    ds = ds.assign_coords({"x": x_coords, "y": y_coords})
    ds = ds.assign_coords({"lat": lat_coords, "lon": lon_coords})

    return ds

In [ ]:
base_url_tempate = (
    "https://carbonplan.blob.core.windows.net/carbonplan-scratch/from_bill/{}"
)
dataset_urls = {
    "Drought": {
        "historical": base_url_tempate.format(
            "Fig1D_AllMortModel_ModeledFIAlongHistMort_3-19-2021.tiff"
        ),
        "observed": base_url_tempate.format(
            "Fig1C_AllMortModel_ObservedHistMort_3-19-2021.tiff"
        ),
    },
    "Insects": {
        "historical": base_url_tempate.format(
            "Fig1F_InsectModel_ModeledFIAlongHistMort_3-26-2021.tif"
        ),
        "observed": base_url_tempate.format(
            "Fig1E_InsectModel_ObservedHistMort_3-26-2021.tif"
        ),
    },
}

## Load in the drought and insect data


#### Observations and Historical simulations


In [ ]:
for mechanism, url_dict in dataset_urls.items():
    for setup, url in url_dict.items():
        print(url)
        ds_dict[mechanism + "_" + setup] = load.tiff(
            url,
            ds_dict["Fire_observed"].lat,
            ds_dict["Fire_observed"].lon,
            ds_dict["Fire_observed"].x.values,
            ds_dict["Fire_observed"].y.values,
        )

## Plot the figure for all three impacts


In [ ]:
full_plot = alt.vconcat().resolve_scale(color="independent")
fire_chart = plot.fire.simple_map(
    ds_dict["Fire_observed"]["monthly"],
    data2=ds_dict["Fire_historical"]["historical"],
    title1="Observed",
    title2="Modeled",
    clim=(0, 0.004),
    clabel="Burn area (fraction/year)",
    cmap=["#F9D3BD", "#E87A3D", "#752003"],  # ["white", "#EA9755"],
)
drought_chart = plot.fire.simple_map(
    ds_dict["Drought_observed"],
    data2=ds_dict["Drought_historical"],
    title1="Observed",
    title2="Modeled",
    clim=(0, 0.02),
    clabel="Drought mortality ()",
    cmap=["#F9C7ED", "#E563BA", "#770361"],
)

insects_chart = plot.fire.simple_map(
    ds_dict["Insects_observed"],
    data2=ds_dict["Insects_historical"],
    title1="Observed",
    title2="Modeled",
    clim=(0, 0.004),
    clabel="Insect morality ()",
    cmap=["#CFE0F9", "#588EF9", "#0432A5"],
)

full_plot &= fire_chart
full_plot &= drought_chart
full_plot &= insects_chart

full_plot.configure_view(strokeOpacity=0)